### Подходы к решению задачи

Распределение на "обычные" и "необычные" происходит следующим образом. Берётся количество шагов за текущий день и сравнивается с интервалом в два стандартных отклонения от среднего количества шагов за последние несколько дней. Как мне кажется, это позволяет нам учитывать изменение количества пройденных шагов за день в течение времени. Чем чаще пользователь проходит примерно равное количество шагов за день, не скорее это количество становится нормой. Тут есть два допущения, которые позволяют делать исключения из основного подхода.

**Память** <br>
Вряд ли среднестатистический пользователь, помнит, сколько шагов он прошел в среду две недели назад. Активность за последние 5 дней, как мне кажется, описывается пользователем лучше. Поэтому, когда я определяю "обычный" день или "необычный" я беру последние 5 дней перед ним и сравниваю активность в текущий день и за пять дней до него.

**Привычка** <br>
Сложно сказать за какое время вырабатывается привычка, для этого  нужно было бы проанализировать дополнительные данные (по режиму сна, пульсу, etc.), поэтому в рамках задания будем считать, что если в течение одного месяца наблюдаются повторяющиеся паттерны движения внутри недели, то такие данные считаются привычными, даже если они отличаются от других дней недели. Например, человек каждый четверг проходит 20000 шагов в течение одного месяца, при этом в другие дни недели в среднем 10000 шагов. В этом случае четверг считается обычным днем, несмотря на то, что количество шагов отличается в два раза от среднего количества в другие дни недели. Кроме того, если в следующем месяце в четверг человек проходит 10000 шагов, то такой день рассматривается как необычный, поскольку сильно отличается от привычки проходить в четверг 20000 шагов.

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import plotly.graph_objs as go
from plotly.subplots import make_subplots
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)

### Подготовка данных

In [2]:
data = pd.read_csv('query_result_2019-12-24T07_29_06.659397Z.csv')
data.shape

(39671, 7)

In [3]:
data.head()

,Time End,Time End Local Tz,Time Offset,Time Start,Time Start Local Tz,User ID,Value
0,2019-05-05T09:40:00+03:00,2019-05-05T12:40:00+03:00,10800,2019-05-05T09:39:59+03:00,2019-05-05T12:39:59+03:00,2166,1.0
1,2019-05-05T11:13:29+03:00,2019-05-05T14:13:29+03:00,10800,2019-05-05T11:12:51+03:00,2019-05-05T14:12:51+03:00,2166,34.0
2,2019-05-05T12:07:42+03:00,2019-05-05T15:07:42+03:00,10800,2019-05-05T12:07:39+03:00,2019-05-05T15:07:39+03:00,2166,1.0
3,2019-05-05T13:04:05+03:00,2019-05-05T16:04:05+03:00,10800,2019-05-05T13:04:02+03:00,2019-05-05T16:04:02+03:00,2166,3.0
4,2019-05-05T13:18:18+03:00,2019-05-05T16:18:18+03:00,10800,2019-05-05T13:18:13+03:00,2019-05-05T16:18:13+03:00,2166,6.0


In [4]:
#количество шагов за каждый день
data['date'] = data['Time Start'].str.split(pat='T',expand=True)[0]
steps = pd.DataFrame(data.groupby('date')['Value'].sum()).reset_index()
steps['date'] = pd.to_datetime(steps['date'])
steps = steps.rename(columns={'Value': 'steps'})

In [5]:
#день недели
days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
steps['day'] = steps['date'].apply(lambda x: x.weekday())
for i,j in enumerate(days):
    steps['day'] = steps['day'].replace({i:j})
steps.shape

(105, 3)

In [6]:
steps.head()

,date,steps,day
0,2019-05-01,21.0,Wednesday
1,2019-05-02,12956.0,Thursday
2,2019-05-03,17003.0,Friday
3,2019-05-04,14312.0,Saturday
4,2019-05-05,11124.0,Sunday


### Графики

<b> Количество шагов в день

In [7]:
fig = go.Figure([go.Scatter(x=steps['date'], y=steps['steps'])])
fig.update_layout(title_text='Количество шагов в день',
                  width=1000, height=500, plot_bgcolor='rgba(0,0,0,0)', showlegend=False,
                  yaxis=dict(gridcolor='#EEEEEE', nticks=15),
                  xaxis=dict(tickangle=-45, gridcolor='#EEEEEE', 
                             tickformat = '%d %B (%a)', nticks=70, tickfont=dict(size=9)))
fig.show()

<b> Декомпозиция временного ряда

In [8]:
from statsmodels.tsa.seasonal import seasonal_decompose
decomposition = seasonal_decompose(steps['steps'], model='multiplicative',freq=1)
trend = decomposition.trend
seasonal = decomposition.seasonal
residual = decomposition.resid

fig = make_subplots(rows=4, cols=1, vertical_spacing = 0.1, shared_xaxes=True,
                    subplot_titles=('Оригинал', 'Тренд', 'Сезонность', 'Остаток'))
fig.add_trace(go.Scatter(x=steps['date'], y=steps['steps']), row=1, col=1)
fig.add_trace(go.Scatter(x=steps['date'], y=trend.values), row=2, col=1)
fig.add_trace(go.Scatter(x=steps['date'], y=seasonal.values), row=3, col=1)
fig.add_trace(go.Scatter(x=steps['date'], y=residual.values), row=4, col=1)

fig.update_layout(width=900, height=500, plot_bgcolor='rgba(0,0,0,0)', showlegend=False)
fig.update_xaxes(tickangle=-45, gridcolor='#EEEEEE', tickformat = '%d %B (%a)', nticks=105, tickfont=dict(size=9), row=4, col=1)
fig.update_xaxes(gridcolor='#EEEEEE',  nticks=105, row=3, col=1)
fig.update_xaxes(gridcolor='#EEEEEE',  nticks=105, row=2, col=1)
fig.update_xaxes(gridcolor='#EEEEEE',  nticks=105, row=1, col=1)
fig.show()

+ <i>  Чёткий тренд и сезонность отстуствуют, значит весь временной интервал для оценки "обычный/необычный" день можно рассматривать как равнозначный

<b> Распределение количества шагов

In [9]:
import plotly.figure_factory as ff
fig = make_subplots(rows=1, cols=2, horizontal_spacing = 0.05, 
                    subplot_titles=('Распределение количества шагов <br> на всём интервале', 'Распределение количества <br> шагов для каждого дня недели')
                   )
fig_dist = ff.create_distplot([steps['steps']], ['Распределение количества шагов'], show_hist=False, show_curve=True, show_rug=False)
fig.append_trace(fig_dist['data'][0], row=1, col=1)

plot_data = []
for i in days:
    day_data = steps[steps['day'] == i]["steps"].values
    plot_data.extend([day_data])

fig_dist_week = ff.create_distplot(plot_data, days, show_rug=False, show_hist=False, show_curve=True)
for i in range(0,7):
    fig.append_trace(fig_dist_week['data'][i], row=1, col=2)

fig.update_layout(width=1000, height=400, plot_bgcolor='rgba(0,0,0,0)', showlegend=False)
fig.update_xaxes(gridcolor='#EEEEEE',  nticks=20, row=1, col=1)
fig.update_xaxes(gridcolor='#EEEEEE',  nticks=20, row=1, col=2)
fig.update_yaxes(gridcolor='#EEEEEE',  nticks=10, row=1, col=2)
fig.update_yaxes(gridcolor='#EEEEEE',  nticks=10, row=1, col=1)
fig.show()

<b> Распределение количества шагов для каждого дня недели

In [10]:
fig = go.Figure()
for i in days:
    y = steps[steps['day'] == i]['steps']
    fig.add_trace(go.Box(y=y, name=i, fillcolor='rgba(0,0,0,0)', line = dict(color='blue')))

fig.update_layout(title_text='Распределение шагов для каждого дня недели',
                  width=900, height=400, plot_bgcolor='rgba(0,0,0,0)',
                  yaxis=dict(gridcolor='#EEEEEE', nticks=10), showlegend=False)
fig.show()    

+ <i>  Выбросы для некоторых дней недели потенциально являются "необычными" днями

<b> Медианное и среднее количество шагов для каждого дня недели

In [11]:
fig = make_subplots(rows=1, cols=2, horizontal_spacing = 0.05, shared_yaxes=True,
                    subplot_titles=('Медианное количество шагов', 'Среднее количество шагов')
                   )
bar_data = steps.groupby(['day'])['steps'].median().sort_values(ascending=True)
fig.add_trace(go.Bar(y=bar_data.index, x=bar_data, width=0.8, orientation='h'), row=1, col=1)

bar_data = steps.groupby(['day'])['steps'].mean()
fig.add_trace(go.Bar(y=bar_data.index, x=bar_data, width=0.8, orientation='h'), row=1, col=2)

fig.update_layout(width=800, height=300, plot_bgcolor='rgba(0,0,0,0)', showlegend=False)

+ <i>  Медианное и среднее количество шагов для каждого дня недели примерно одинаковое. Нельзя сказать, что на наблюдаемом интервале дни недели как-то особенно отличаются друг от друга

<b>Количество шагов в будни и выходные

In [12]:
fig = go.Figure()
fig.add_trace(go.Bar(x=steps[steps['day'].isin(['Sunday','Saturday'])]['date'], 
                     y=steps[steps['day'].isin(['Sunday','Saturday'])]['steps'],
                     marker_color='yellowgreen',
                     name='Выходные'))
fig.add_trace(go.Bar(x=steps[~steps['day'].isin(['Sunday','Saturday'])]['date'], 
                     y=steps[~steps['day'].isin(['Sunday','Saturday'])]['steps'],
                     marker_color='blue',
                     name='Будни')) 
fig.update_layout(title_text='Количество шагов в будни и выходные',
                  width=1000,
                  height=400,
                  plot_bgcolor='rgba(0,0,0,0)',
                  xaxis=dict(tickangle=-45, nticks=105, tickformat = '%d %B -%a', tickfont=dict(size=8)),
                  yaxis=dict(gridcolor='#EEEEEE'))
fig.show()

# Функция разметки дней

Определяем какой сегодня день "обычный", "необычный" или "неизвестно", анализируем историю шагов и привычки

In [13]:
def define_day(data):
    
    #предобработка данных
    data['date'] = data['Time Start'].str.split(pat='T',expand=True)[0]
    data = pd.DataFrame(data.groupby('date')['Value'].sum()).reset_index()
    data['date'] = pd.to_datetime(data['date'])
    days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
    data['day'] = data['date'].apply(lambda x: x.weekday())
    data = data.rename(columns={'Value': 'steps'})
    for i,j in enumerate(days):
        data['day'] = data['day'].replace({i:j})
    
    #определяем день по истории шагов за последние 5 дней    
    temp = data.copy()
    temp['past_std'] = temp['steps'].rolling(5).std().shift()
    temp['past_mean'] = temp['steps'].rolling(5).mean().shift()

    def day_type(row):
        mean = row['past_mean']
        std = row['past_std']
        if row['steps'] >= mean - std or row['steps'] <= mean + std:
             row['day_type'] = 'обычный день'
        if row['steps'] < mean - std or row['steps'] > mean + std:
             row['day_type'] = 'необычный день'
        if pd.isna(row['past_mean']) == True:
            row['day_type'] = 'неизвестно'
        return row['day_type']
    
    temp['day_type'] = ''
    temp['day_type'] = temp.apply(day_type, axis=1)
    
    #поиск привычек связанных с днями недели
    temp['habit'] = ''
    for i in days:
        std = temp[temp['day'] == i]['steps'].rolling(4).std().shift()
        mean = temp[temp['day'] == i]['steps'].rolling(4).mean().shift()
        habit = std/mean
        temp['habit'][habit.index] = habit.values
    
    #проверяем день недели с учетом привычки за последние 4 недели    
    for i in range(0, temp.shape[0]):
        day = temp.iloc[i]['day']
        past_days = temp[(temp.index <=i) & (temp['day'] == day)].index[-5:-1].to_list()
        habit = (temp['steps'][i] - temp['steps'][past_days].mean()) / temp['steps'][past_days].mean()
    
        mean = temp['past_mean'][i]
        std = temp['past_std'][i]
        uncommom = temp['steps'][i] < (mean - std) or temp['steps'][i] > (mean + std)
        
        #здесь проверяем что привычка сохраняется (отклонение от предыдущих 4 недель до 10%) минимум 4 недели подряд
        if np.abs(habit) <= 0.1 and temp['habit'][i] <=0.1 and uncommom == True and len(past_days) == 4:
            temp['day_type'][i]='обычный день'
        elif np.abs(habit) > 0.1 and temp['habit'][i] <=0.1 and len(past_days) == 4:
            temp['day_type'][i]='необычный день'
    data['day_type'] = temp['day_type']
    return data

In [14]:
result = define_day(data)
result.head(10)

,date,steps,day,day_type
0,2019-05-01,21.0,Wednesday,неизвестно
1,2019-05-02,12956.0,Thursday,неизвестно
2,2019-05-03,17003.0,Friday,неизвестно
3,2019-05-04,14312.0,Saturday,неизвестно
4,2019-05-05,11124.0,Sunday,неизвестно
5,2019-05-06,7467.0,Monday,обычный день
6,2019-05-07,9027.0,Tuesday,обычный день
7,2019-05-08,12888.0,Wednesday,обычный день
8,2019-05-09,12982.0,Thursday,обычный день
9,2019-05-10,26759.0,Friday,необычный день


In [25]:
def bar_plot(data):
    fig = go.Figure()
    colors = ['lightgrey','blue','red']
    for i,j in enumerate(['неизвестно', 'обычный день', 'необычный день']):
        fig.add_trace(go.Bar(x=data[data['day_type'] == j]['date'], 
                             y=data[data['day_type'] == j]['steps'],
                             marker_color=colors[i],
                             name=j))
    fig.update_layout(title_text='Размеченные данные',
                      width=1000,
                      height=400,
                      plot_bgcolor='rgba(0,0,0,0)',
                      xaxis=dict(tickangle=-45, nticks=105, tickformat = '%d %B -%a', tickfont=dict(size=8)),
                      yaxis=dict(gridcolor='#EEEEEE'))
    fig.show()

In [26]:
bar_plot(result)

# Проверка

<b> Проверим работу функции на сгенерированных данных с учетом наличия недельных паттернов

In [27]:
#подготовка данных для обработки
test = data[['Time Start','Value','date']][0:105]
test['Time Start'] = result['date'].astype(str)
test['Value'] = result['steps']

In [28]:
#генерируем случайные данные и привычки по дням недели
test['Value'] = np.random.randint(5000, 15000, size=test.shape[0])
for i in ['2019-05-27','2019-06-03','2019-06-10','2019-06-17','2019-06-24']:
    test['Value'].iloc[test[test['Time Start'] == i].index] = 20000
test['Value'][test[test['Time Start'] =='2019-07-01'].index] = test[test['Time Start'] =='2019-06-30']['Value']

In [29]:
bar_plot(define_day(test))

<i> С 27 мая по 17 июня пользователь выработал привычку проходить 20 тысяч шагов каждый день по понедельникам. Первые четыре недели такие дни считались необычными. На пятую неделю привычка закрепилась, поэтому 24 июня — обычный день. Ещё через неделю пользователь прошел необычно мало, по сравнению с привычным для него количеством шагов в понедельник. Поэтому несмотря на то, что 30 июня и 1 июля человек прошел равное количество шагов, 30 июня считается необычным днём, так как это понедельник, за который последние 5 недель он обычно проходил 20 тысяч шагов.

# Выводы

+ "Обычными днями" считаются дни, когда сумма шагов за день попадает в диапазон mean-std/mean+std на временном интервале 5 дней до даты оцениваемого дня. Выходящие за этот диапазон значения маркируются как "необычные дни".
<br>
     
+ Когда пользователь только начал измерять количество шагов, нельзя сказать о днях обычные они или необычные, поэтому первые несколько дней маркируются как неизвестные.
<br>

+ Приведенная выше функция маркировки дней, как мне кажется, имеет один важный недостаток: деление на "обычные" и "необычные" основано на попадании/не попадании в определенный интервал — на границе этого интервала могут быть обычные и необычные дни с мало отличающимся количеством шагов, что может вводить пользователя в заблуждение.
<br>

+ Функция не учитывает возможную летнюю/зимнюю сезонность. Дополнительные годовые данные помогли бы с этой проблемой справиться.
<br>

+ Функция не учитывает возрастающие тренды. Если пользователь занимается спортом, постепенно увеличивая нагрузку, допустим, в течение месяца, то день со сниженным количеством шагов должен считаться необычным, даже если сумма шагов за день находится в пределах диапазона в одно стандартное отклонение за последние 5 дней.
<br>

+ В силу того, что акцент сделан на количестве шагов за день, не учтены паттерны передвижения внутри дня. 
<br>

+ Допущения, принятые в расчетах (привычка, память) могли бы быть сняты при анализе допополнительных данных о режимах сна, пульсе, питании etc.

    